Compare the differences in the Nash Bargaining solution, grouped by Issue and p-value

Set the variable `model_runs` to your corresponding database id's 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import ipywidgets as widget
from decideanalyses import settings
%matplotlib inline

TypeError: descriptor '__subclasses__' of 'type' object needs an argument

In [ ]:
conn = sqlite3.connect(settings.DATABASE_URL)
data_set_id = settings.DATA_SET_ID
model_runs = [28, 29]

In [ ]:
df = pd.read_sql("""
SELECT
  a.p as p,
  a.issue as issue,
  a.iteration as iteration,
  a.repetion as repetion,
  a.numerator / a.denominator AS nbs
FROM (SELECT
        sum(ai.position * ai.power * ai.salience) AS numerator,
        sum(ai.salience * ai.power)               AS denominator,
        r.pointer                                 AS repetion,
        i2.pointer                                AS iteration,
        m.p,
  i.name as issue
      FROM actorissue ai
        LEFT JOIN issue i ON ai.issue_id = i.id
        LEFT JOIN actor a ON ai.actor_id = a.id
        LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
        LEFT JOIN repetition r ON i2.repetition_id = r.id
        LEFT JOIN modelrun m ON r.model_run_id = m.id        
      WHERE  ai.type = 'before' AND m.id IN (%s)
     GROUP BY m.id,r.id, i2.id, i.id) a
""" % ','.join(['13', '14']), 
                 conn, 
                 # params=((13, 14), ),
                 index_col='p',                  
                 columns=['nbs']
                )

def avg_std(series):
    avg = np.average(series)
    std = np.var(series, ddof=1)
    
    return '{0} ({1})'.format(round(avg, 5), round(std, 5)) 

table_avg = pd.pivot_table(df, index=['issue','p'], columns=['iteration'],values=['nbs'],aggfunc=np.average)
table_var = pd.pivot_table(df, index=['issue','p'], columns=['iteration'],values=['nbs'],aggfunc=np.var)


# Verloop NBS
In onderstaande tabel wordt het gemiddelde verloop van de NBS scores getoond per issue. Tussen haakjes achter de NBS score staat de standaarddeviatie.

Onder de tabel staat het verloop van het gemiddelde afgebeeld in een grafiek per issue

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

table_avg

In [ ]:
table_var

In [ ]:
cursor = conn.execute('SELECT issue.name, issue.id FROM issue WHERE issue.data_set_id = ? ORDER BY issue.name', [data_set_id])
issues = cursor.fetchall()

In [ ]:
for name, issue_id in issues:
    df = pd.read_sql("""
    SELECT
    a.p as p,
    a.issue as issue,
    a.iteration as iteration,
    a.repetion as repetion,
    a.numerator / a.denominator AS nbs
    FROM (SELECT
    sum(ai.position * ai.power * ai.salience) AS numerator,
    sum(ai.salience * ai.power)
    AS denominator,
    r.pointer
    AS repetion,
    i2.pointer
    AS iteration,
    m.p,
    i.name as issue
    FROM actorissue ai
    LEFT JOIN issue i ON ai.issue_id = i.id
    LEFT JOIN actor a ON ai.actor_id = a.id
    LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
    LEFT JOIN repetition r ON i2.repetition_id = r.id
    LEFT JOIN modelrun m ON r.model_run_id = m.id
    LEFT JOIN dataset d ON a.data_set_id = d.id
    WHERE ai.type = 'before' AND d.id = ? AND i.id = ?
    GROUP BY m.id,r.id, i2.id, i.id) a
    """,
    conn,
    params=(1,issue_id),
    index_col='p',
    columns=['nbs']
    )
    table = pd.pivot_table(df, index=['iteration'], columns=['p'], values=['nbs'])
    plot = table.plot()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title(name)
    plt.ylim(0, 110)
    plt.show()